#Задание 9(b)

##Критерий $\chi^2$:

In [2]:
from scipy.integrate import quad
from scipy import optimize
import numpy as np
import scipy.stats


def integrate_normal(lower, upper, teta1, teta2):
  return quad(lambda x: 1/(np.sqrt(2 * np.pi) * teta2) * np.e ** (-(x - teta1) ** 2 / (2 * teta2 ** 2)), lower, upper)[0]

def func_to_optimize(tetas):
  res = 1
  selection = np.array([5, 8, 6, 12, 14, 18, 11, 6, 13, 7])
  for i in range(10):
    if(i == 0):
      res *= integrate_normal(- np.inf, 1, tetas[0], tetas[1]) ** selection[i]
    elif(i == 9):
      res *= integrate_normal(9, np.inf, tetas[0], tetas[1]) ** selection[i]
    else:
      res *= integrate_normal(i, i + 1, tetas[0], tetas[1]) ** selection[i]
  return -res



x0 = np.array([11, 13])
maximized = optimize.minimize(func_to_optimize, x0, method = 'nelder-mead')
selection = np.array([5, 8, 6, 12, 14, 18, 11, 6, 13, 7])
delta_wave = 0
tetas = maximized.x
print(tetas, "- Теты, при которых достигается максимум")
for i in range(10):
  if(i == 0):
    integral = integrate_normal(- np.inf, 1, tetas[0], tetas[1]) * 100
    delta_wave += (selection[i] - integral) ** 2 / integral
    print(integral)
  elif(i == 9):
    integral = integrate_normal(9, np.inf, tetas[0], tetas[1]) * 100
    delta_wave += (selection[i] - integral) ** 2 / integral
    print(integral)
  else:
    integral = integrate_normal(i, i + 1, tetas[0], tetas[1]) * 100
    delta_wave += (selection[i] - integral) ** 2 / integral
    print(integral)
print("Дельта с волной:", delta_wave)

print(scipy.stats.chi2(7).sf(9.8), "= p-value")

[5.28962548 2.67951583] - Теты, при которых достигается максимум
5.470000660686341
5.508092515463962
8.663522214644964
11.873887426058692
14.180762281175316
14.757614759419813
13.382679980262724
10.574976112618318
7.281532461692063
8.306931587977822
Дельта с волной: 9.802654592391548
0.2001934364129521 = p-value


##Критерий Колмогорова:

In [3]:
selection = np.array([5, 8, 6, 12, 14, 18, 11, 6, 13, 7])

def find_parameter_mean(x):
  return np.mean(x)

def find_parameter_variance(slection_1):
  return np.sqrt(np.var(slection_1) * 100 / 99)

def count_delta(selection, teta1, teta2):
  mx = 0
  incr_selection = [0] * 10
  incr_selection[0] = 0
  for i in range(1, len(selection)):
    incr_selection[i] = selection[i - 1] + incr_selection[i - 1]
  for i in range(len(incr_selection) - 1):
    integral = scipy.stats.norm(teta1, teta2).cdf(i)
    if i != 9:
      if mx < max(abs(integral - incr_selection[i] / 100), abs(integral - incr_selection[i + 1] / 100)):
        mx = max(abs (integral - incr_selection[i] / 100), abs(integral - incr_selection[i + 1] / 100))
    else:
      if mx < abs(integral - incr_selection[i] / 100):
        mx = abs(integral - incr_selection[i] / 100)
  print(10 * mx)
  return 10 * mx

def bootstrap_par(teta1_waved, teta2_waved, n_bootstrap_samples=50000):
  bootstrap_deltas = []
  for i in range(n_bootstrap_samples):
    samples = np.random.normal(teta1_waved, teta2_waved, 100).astype(int)
    selection_intervals = [0] * 10
    for k in samples:
      if k < 0:
        k = 0
      elif k > 9:
        k = 9
      selection_intervals[k] += 1
    bootstrap_deltas.append(count_delta(selection_intervals, find_parameter_mean(samples), find_parameter_variance(samples)))
    print(i)
  return bootstrap_deltas


sel = []
for i in range(len(selection)):
  for j in range(selection[i]):
    sel.append(i)
teta1 = find_parameter_mean(sel)
teta2 = find_parameter_variance(sel)
delta = count_delta(selection, teta1, teta2)
print(selection)
print(delta, "- дельта для исходной выборки")
deltas = bootstrap_par(teta1, teta2)
deltas.sort()
s = 0
for i in range(50000):
  if delta > deltas[i]:
    s += 1
  else:
    break
print()

print(1 - s / 50000, end = " ")
if 1 - s / 50000 <= 0.05:
  print("Отбрасываем")
elif 1 - s / 50000 > 0.05:
  print("Не имеем оснований отбросить")



Выходные данные были обрезаны до нескольких последних строк (5000).
1.1720432919074664
47501
1.380319121799463
47502
1.0267150925544293
47503
1.312696487916758
47504
1.1069173871958693
47505
1.542102709400191
47506
1.3803219705493692
47507
1.2538966129574054
47508
0.9809623418882574
47509
1.0028018434911923
47510
0.9497918466511202
47511
1.5738042263824243
47512
0.9906385055627609
47513
1.34162287457267
47514
1.210025836478073
47515
1.389479864936869
47516
1.322083153240735
47517
1.179438985737482
47518
1.5667606846068949
47519
1.189200645621038
47520
1.0736954806624426
47521
1.128644883791396
47522
1.4299018746076542
47523
1.2833766575743233
47524
0.9190096250199281
47525
1.3221031922641875
47526
1.1275588821699745
47527
0.88505484468827
47528
0.9902405637464307
47529
1.3740174409047023
47530
0.9440077905215483
47531
1.525838739168185
47532
1.1337326243201429
47533
0.9966256279296559
47534
1.1479311325963053
47535
1.1319468285189271
47536
1.009652592561141
47537
1.0152448850316254
475